# Using Redis and Azure OpenAI to chat with PDF documents

This notebook demonstrates how to use RedisAI and Azure OpenAI to chat with PDF documents. The PDF included is
a informational documents about AI / ML published by SDAIA.

In this notebook, we will use LLamaIndex to chunk, vectorize, and store the PDF document in Redis as vectors
alongside associated text. The query interface provided by LLamaIndex will be used to search for relevant
information given queries from the user.

In [ ]:
# Install the requirements
%pip install redis pypdf PyPDF2 python-dotenv transformers tiktoken llama_index==0.6.8

In [1]:
import os
import textwrap
import openai
from langchain.llms import AzureOpenAI, OpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index.vector_stores import RedisVectorStore
from llama_index import LangchainEmbedding
from llama_index import (
    GPTVectorStoreIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    PromptHelper,
    ServiceContext,
    StorageContext
)
import sys

import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO) # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [2]:
# load the .env file in the parent directory into the current environment
from dotenv import load_dotenv
load_dotenv('./.env')

True

# Azure OpenAI 

The notebook allows the user to use Azure OpenAI endpoints. Make sure to follow the instructions in the README and set the .env correctly according to Key and Endpoint from Portal Azure API you are using. 

## Azure OpenAI 

Here we setup the AzureOpenAI models and API keys that we set by reading from the environment above. The ``PromptHelper`` sets the parameters for the OpenAI model. The classes defined here are used together to provide a QnA interface between the user and the LLM.

In [3]:
# setup Llama Index to use Azure OpenAI
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_API_BASE")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

# Get the OpenAI model names ex. "text-embedding-ada-002"
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
text_model = os.getenv("OPENAI_TEXT_MODEL")


print(f"Using models: {embedding_model} and {text_model}")

# get the Azure Deployment name for the model
embedding_model_deployment = os.getenv("AZURE_EMBED_MODEL_DEPLOYMENT_NAME")
text_model_deployment = os.getenv("AZURE_TEXT_MODEL_DEPLOYMENT_NAME")

print(f"Using deployments: {embedding_model_deployment} and {text_model_deployment}")


Using models: text-embedding-ada-002 and gpt-35-turbo
Using deployments: embedding-model and gpt35-model


In [4]:

llm = AzureOpenAI(deployment_name=text_model_deployment, model_kwargs={
    "api_key": openai.api_key,
    "api_base": openai.api_base,
    "api_type": openai.api_type,
    "api_version": openai.api_version,
})
llm_predictor = LLMPredictor(llm=llm)

embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model=embedding_model,
        deployment=embedding_model_deployment,
        openai_api_key= openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)

### LLamaIndex

[LlamaIndex](https://github.com/jerryjliu/llama_index) (GPT Index) is a project that provides a central interface to connect your LLM's with external data sources. It provides a simple interface to vectorize and store embeddings in Redis, create search indices using Redis, and perform vector search to find context for generative models like GPT.

Here we will use it to load in the documents (Chevy Colorado Brochure).

In [14]:
# load documents
documents = SimpleDirectoryReader('./docs').load_data()
print('Document ID:', documents[0].doc_id)

Documents Count:  1038


Llamaindex also works with frameworks like langchain to make prompting and other aspects of a chat based application easier. Here we can use the ``PromptHelper`` class to help us generate prompts for the (Azure) OpenAI model. The will be off by default as it can be tricky to setup correctly.

In [6]:
# set number of output tokens
num_output = int(os.getenv("OPENAI_MAX_TOKENS"))
# max LLM token input size
max_input_size = int(os.getenv("CHUNK_SIZE"))
# set maximum chunk overlap
max_chunk_overlap = int(os.getenv("CHUNK_OVERLAP"))

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [7]:

# define the service we will use to answer questions
# if you executive the Azure OpenAI code above, your Azure Models and creds will be used and the same for OpenAI
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embedding_llm,
#    prompt_helper=prompt_helper # uncomment to use prompt_helper.
)

## Initialize Redis as a Vector Database

Now we have our documents read in, we can initialize the ``RedisVectorStore``. This will allow us to store our vectors in Redis and create an index.

The ``GPTVectorStoreIndex`` will then create the embeddings from the text chunks by calling out to OpenAI's API. The embeddings will be stored in Redis and an index will be created.

NOTE: If you didn't set the ``OPENAI_API_KEY`` environment variable, you will get an error here.

In [8]:
def format_redis_conn_from_env(using_ssl=False):
    start = "rediss://" if using_ssl else "redis://"
    # if using RBAC
    password = os.getenv("REDIS_PASSWORD", None)
    username = os.getenv("REDIS_USERNAME", "default")
    if password != None:
        start += f"{username}:{password}@"

    return start + f"{os.getenv('REDIS_ADDRESS')}:{os.getenv('REDIS_PORT')}"


# make using_ssl=True to use SSL with ACRE
redis_address = format_redis_conn_from_env(using_ssl=False)

print(f"Using Redis address: {redis_address}")
vector_store = RedisVectorStore(
    index_name="chevy_docs",
    index_prefix="blog",
    redis_url=redis_address,
    overwrite=True
)

# access the underlying client in the RedisVectorStore implementation to ping the redis instance
vector_store.client.ping()

Using Redis address: redis://default:@localhost:6379


True

In [9]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context
)

Token indices sequence length is longer than the specified maximum sequence length for this model (1179 > 1024). Running this sequence through the model will result in indexing errors


INFO:llama_index.vector_stores.redis:Deleting index chevy_docs
Deleting index chevy_docs
INFO:llama_index.vector_stores.redis:Creating index chevy_docs
Creating index chevy_docs
INFO:llama_index.vector_stores.redis:Added 1933 documents to index chevy_docs
Added 1933 documents to index chevy_docs
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 1483011 tokens
> [build_index_from_nodes] Total embedding token usage: 1483011 tokens


## Start Querying information from the Document

Now that we have our document stored in the index, we can ask questions against the index. The index will use the data stored in itself as the knowledge base for chatgpt.

In [15]:
query_engine = index.as_query_engine()
response = query_engine.query("لمحة عن جهود ألمانيا في تطوير الذكاء الاصطناعي")
print("\n", textwrap.fill(str(response), 100))

INFO:llama_index.vector_stores.redis:Querying index chevy_docs
Querying index chevy_docs
INFO:llama_index.vector_stores.redis:Found 2 results for query with id ['blog_d1184027-d6c2-492b-9121-8d8d369388dc', 'blog_6f3f67e1-2c5f-46f1-b858-02419df98d9d']
Found 2 results for query with id ['blog_d1184027-d6c2-492b-9121-8d8d369388dc', 'blog_6f3f67e1-2c5f-46f1-b858-02419df98d9d']
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 47 tokens
> [retrieve] Total embedding token usage: 47 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2482 tokens
> [get_response] Total LLM token usage: 2482 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens

 الذكاء الاصطناعي (AI) والابتكار جز

In [17]:
response = query_engine.query("ما الجهود التي بذلتها الصين في تطوير الذكاء الاصطناعي?")
print("\n", textwrap.fill(str(response), 100))

INFO:llama_index.vector_stores.redis:Querying index chevy_docs
Querying index chevy_docs
INFO:llama_index.vector_stores.redis:Found 2 results for query with id ['blog_d1184027-d6c2-492b-9121-8d8d369388dc', 'blog_fb789e7d-32ba-4483-b4b2-b83589500f75']
Found 2 results for query with id ['blog_d1184027-d6c2-492b-9121-8d8d369388dc', 'blog_fb789e7d-32ba-4483-b4b2-b83589500f75']
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 51 tokens
> [retrieve] Total embedding token usage: 51 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2469 tokens
> [get_response] Total LLM token usage: 2469 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens

 استخدام الذكاء الاصطناعي ونظم دعم 

In [18]:
response = query_engine.query("ماهي الجهود الوطنية في مجال الذكاء الاصطناعي?")
print("\n", textwrap.fill(str(response), 100))

INFO:llama_index.vector_stores.redis:Querying index chevy_docs
Querying index chevy_docs
INFO:llama_index.vector_stores.redis:Found 2 results for query with id ['blog_6f3f67e1-2c5f-46f1-b858-02419df98d9d', 'blog_84c122d7-bf27-4422-b1c7-140e6d03083c']
Found 2 results for query with id ['blog_6f3f67e1-2c5f-46f1-b858-02419df98d9d', 'blog_84c122d7-bf27-4422-b1c7-140e6d03083c']
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 41 tokens
> [retrieve] Total embedding token usage: 41 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1850 tokens
> [get_response] Total LLM token usage: 1850 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens

 الجهود الوطنية في مجال الذكاء الاص

In [12]:
llm('عطيني نكتة')

'")\n\n# C:\\Python27\\python.exe -u "c:\\Users\\Ahmed\\Desktop\\python\\if_elif_else.py"\n# مرحبا بك بلعبة النكت\n# أكتب نكتة: \n# محشش سأل أبوه: البنزين بيتحرق بالموتر؟ قال: ياولدي البنزين بيتحرق بالموتر.. لكن اللي يتحرق هو الموتر\n# النكتة مضحكة جدا! هههههههههههههه\n# مرحبا بك بلعبة النكت\n# أكتب نكتة: \n# نكته اخري\n# النكتة مضحكة جدا! هههههههههههههه\n# مرحبا بك بلعبة النكت\n# أكت'

In [16]:
llm_result = llm.generate(["عطيني نكتة", "خبرني نكتبة"]*15)

In [17]:
len(llm_result.generations)

30

In [18]:
llm_result.generations[0]

[Generation(text=' احلى من الثانية😂😂😂\n\nليش الشايب بيحب الحرب ؟؟؟؟؟؟؟؟؟؟ لانه بيشرب شاي البندقية 😂😂😂\n\nبيقولك طالب مش عارف يتكلم انجليزي قعد يتعلم اللهجات وبعد سنتين قابل انجليزي قعد يتكلم اللهجات بس انجليزي بنطقها صح\n\nزوجة تقول لزوجها: اذا بتعرف انا ايش اسمي اطلقني؟؟؟؟؟ قالها : موضوع مش مطروح السيدة😂😂😂😂\n\nاستاذ بحكيلي عن مرة', generation_info={'finish_reason': 'length', 'logprobs': None})]

In [11]:
response

Response(response="{'question': 'عطني لمحة تاريخية عن تطور النماذج للغوية الكبيرة', 'context': 'افتتاحية\\nمــر العالــم بتحــولات كبيــرة خــال العاميــن الماضييــن، فمــع زوال الجائحــة والعــودة إلــى الوضــع الطبيعــي \\nلــم تعــد كل الأشــياء إلــى طبيعتهــا، فقــد شــهد العالــم تقدمــاً... الاصطناعــي خ", source_nodes=[NodeWithScore(node=Node(text='page_label: 50\n\nسياق النص، وأن الأمر أعمق وأبعد من بناء نموذج يقيس الروابط بين النصوص فقط، وأكد أننا ما زلنا بحاجة إلى نموذج كما ناقش الحاجة إلى معايير قياس دقيقة تساعد على معرفة مستوى تقدم الآلة في محاكاتها للبشر بإشراك علماء النفس الإدراكي وعلماء الأعصاب الذين يحاولون فهم آلية التعلم لدى البشر أثناء بناء النماذج. التعبير عنهما ببعض الجمل الناقلة، كما يجب عدم إهمال تمثيل الإحساس والمعنى في اللغة لذا يوصي الموثوقة، و أنه يجب فهم التعقيدات الكبيرة خلف اللغة البشرية التي تحوي على معنى ومضمون يراد والمشرعين لسامة وأخاقية الاستخدام، وأشار إلى الحرص على تتبع المعلومات الدقيقة من المصادر \nومن ثم بناء نماذج لغوية جديرة بالثقة.ع\n', doc_i